# Preprocessing
## Data 다루기

### 데이터 전처리의 목적

데이터의 품질이 목적에 맞는지 확인하고 필요하면 품질을 높이는 작업

. 데이터를 목적에 맞게 변경

. 데이터 압축
- 데이터의 특징을 정확히 알고 있을 때 목적에 꼭 필요한 부분만을 사용
- 시스템의 부하를 줄이기 위해 사용

----

**사용된 예제 데이터**

1. Photo diode 신호 (`./Data/PD_Data.csv`)

- 50000 Hz
- 0.01 s
- 500 EA
- Data size (, 500)

<img src="image1.png" alt="Drawing" style="width: 500px;"/>

2. Weld pool 동기화 촬영 이미지 (`./Data/image/*.jpg`)

- 10000 Hz
- 0.01 s
- Mono(gray image)
- Image size (232, 768)
- 100 EA

<img src="image2.png" alt="Drawing" style="width: 500px;"/>
<img src="image3.png" alt="Drawing" style="width: 500px;"/>

----

In [11]:
# Photo Diode data 불러오기
import numpy as np

_PD_path = "./PD_Data.csv"
PD_data = np.loadtxt(_PD_path, delimiter=',', dtype=np.str, encoding='utf-8')
PD_data = np.array(PD_data, dtype=np.float32)
print(PD_data.shape)

(500,)


In [12]:
# image data 불러오기
import glob
import cv2

_img_path = './image/*.jpg'
_imgs_path = sorted(glob.glob(_img_path))

imgs = []
for i in _imgs_path:
    
    imgs.append(cv2.imread(i, cv2.IMREAD_GRAYSCALE))

imgs = np.array(imgs)
print(imgs.shape)

(100, 232, 768)


## 데이터 처리 예제

### 데이터 싱크 맞추기

Photo diode 신호와 Weld pool 동기화 촬영 이미지를 1000 Hz로(0.001 s)

### 어떻게?
1. Photo diode 신호
- 50000 Hz to 1000 Hz
- 50개 데이터 당 하나의 데이터(0.0005 s 간의 시계열 데이터)
- (10, 50)

<img src="image4.png" alt="Drawing" style="width: 500px;"/>


2. Weld pool 동기화 촬영 이미지
- 10000 Hz to 1000 Hz
- 10개 이미지 당 하나의 데이터
- 10개 이미지의 각 픽셀의 intensity의 평균
- (10, 232, 768)

<img src="image5.png" alt="Drawing" style="width: 500px;"/>


In [13]:
# Photo Diode data 나누기 1 - 반복문

PD_data2 = np.zeros((10, 50))

for i in range(10):
    PD_data2[i] = PD_data[i*50:i*50+50]
    print(i*50,'부터',i*50+50,'까지 저장 완료')

print(PD_data2.shape)
#print(PD_data2)

0 부터 50 까지 저장 완료
50 부터 100 까지 저장 완료
100 부터 150 까지 저장 완료
150 부터 200 까지 저장 완료
200 부터 250 까지 저장 완료
250 부터 300 까지 저장 완료
300 부터 350 까지 저장 완료
350 부터 400 까지 저장 완료
400 부터 450 까지 저장 완료
450 부터 500 까지 저장 완료
(10, 50)


In [14]:
# Photo Diode data 나누기 2 - 함수를 알고 있다면

PD_data3 = PD_data.reshape((10,50))

print(PD_data3.shape)
#print(PD_data3)

(10, 50)


In [15]:
# 만든 데이터 저장

np.savetxt('Output.csv',PD_data3,delimiter=",")

In [16]:
# image data 합성하기 1 

mean_imgs = np.zeros((10,232,768), dtype = np.uint8)

for i in range(10):
        mean_imgs[i] = np.mean(imgs[i*10:i*10+10], axis=0)
        print(i*10,'부터',i*10+10,'까지 저장 완료')
        
print(mean_imgs.shape)
#print(mean_imgs)

0 부터 10 까지 저장 완료
10 부터 20 까지 저장 완료
20 부터 30 까지 저장 완료
30 부터 40 까지 저장 완료
40 부터 50 까지 저장 완료
50 부터 60 까지 저장 완료
60 부터 70 까지 저장 완료
70 부터 80 까지 저장 완료
80 부터 90 까지 저장 완료
90 부터 100 까지 저장 완료
(10, 232, 768)


In [17]:
# image data 합성하기 2 - 변수의 활용

avrV = 10
cols, rows = imgs[0].shape
mean_imgs = np.zeros((int(imgs.shape[0]/avrV),cols,rows), dtype = np.uint8)

for i in range(int(imgs.shape[0]/avrV)):
        mean_imgs[i] = np.mean(imgs[i*avrV:i*avrV+avrV], axis=0)
        print(i*avrV,'부터',i*avrV+avrV,'까지 저장 완료')
        
print(mean_imgs.shape)
#print(mean_imgs)

0 부터 10 까지 저장 완료
10 부터 20 까지 저장 완료
20 부터 30 까지 저장 완료
30 부터 40 까지 저장 완료
40 부터 50 까지 저장 완료
50 부터 60 까지 저장 완료
60 부터 70 까지 저장 완료
70 부터 80 까지 저장 완료
80 부터 90 까지 저장 완료
90 부터 100 까지 저장 완료
(10, 232, 768)


In [18]:
# 폴더 만들기

import os
 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder('./Output')

In [19]:
# 만든 이미지 저장
# mean_imgs.shape[0] = 10
for i in range(mean_imgs.shape[0]):
    cv2.imwrite('./Output/'+str(i).zfill(4)+'.jpg',mean_imgs[i])

## 안내
- 본 자료는 아래의 사람들이 만들었습니다.
    유현정 (Portland State University)
    이기동 (Portland State University)
    김철희 (한국생산기술연구원, Portland State University)

- 예제 파일은 아래에서 받을 수 있습니다.
     https://deepjoin.github.io/dl/

- 문의사항 및 의견: deepjoining@gmail.com

자료는 한국생산기술연구원 용접접합그룹 신입대학원생 교육자료입니다.
일체의 다른 용도 사용을 금지합니다.
